# Test Yolo Inference

In [1]:
import services.ai_services.my_models as Models
import services.image_handler.utils as IMUtils
import services.ai_services.inferences as INF

In [ ]:
yolo = Models.YOLOv8(Models.YoloType.Custom.Plate_best)

In [ ]:
image = IMUtils.read_image('C:/Users/Alireza/Desktop/plate/2171038_FRONT.jpg')
image_rgb = IMUtils.convert_to_rgb(image)
image2 = IMUtils.read_image('C:/Users/Alireza/Desktop/plate/2170593_FRONT.jpg')
image_rgb2 = IMUtils.convert_to_rgb(image2)
IMUtils.show_image(image_rgb)
IMUtils.show_image(image_rgb2)
images = [image, image2]

In [6]:
inference = INF.YOLOInference(yolo)

In [ ]:
cropped_out = inference.run_full_pipeline(images)
for i in cropped_out:
    IMUtils.show_image(i, turn_grey=True, cmap='grey')

# Get and Save Plates

In [1]:
import services.image_handler.utils as IMUtils
import services.ai_services.inferences as INF
import services.ai_services.my_models as Models

In [2]:
images = IMUtils.read_images_from_file("C:/Users/Alireza/Desktop/cars/")

Reading images: 100%|██████████| 139/139 [00:02<00:00, 64.51file/s]


In [4]:
model = Models.YOLOv8(Models.YoloType.Custom.Plate_best)
inference = INF.YOLOInference(model)

Loading YOLO model from weights/best(plate).pt ...

Layer (type:depth-idx)                                       Param #
YOLO                                                         --
├─DetectionModel: 1-1                                        --
│    └─Sequential: 2-1                                       --
│    │    └─Conv: 3-1                                        (1,392)
│    │    └─Conv: 3-2                                        (41,664)
│    │    └─C2f: 3-3                                         (111,360)
│    │    └─Conv: 3-4                                        (166,272)
│    │    └─C2f: 3-5                                         (813,312)
│    │    └─Conv: 3-6                                        (664,320)
│    │    └─C2f: 3-7                                         (3,248,640)
│    │    └─Conv: 3-8                                        (1,991,808)
│    │    └─C2f: 3-9                                         (3,985,920)
│    │    └─SPPF: 3-10                       

In [5]:
cropped_out = inference.run_full_pipeline(images)

Running full pipeline... 

conf=0.5, show=False, save=False
Running inference on Image

0: 640x640 1 plate, 19.2ms
1: 640x640 1 plate, 19.2ms
2: 640x640 1 plate, 19.2ms
3: 640x640 1 plate, 19.2ms
4: 640x640 2 plates, 19.2ms
5: 640x640 8 plates, 19.2ms
6: 640x640 6 plates, 19.2ms
7: 640x640 1 plate, 19.2ms
8: 640x640 1 plate, 19.2ms
9: 640x640 1 plate, 19.2ms
10: 640x640 2 plates, 19.2ms
11: 640x640 1 plate, 19.2ms
12: 640x640 1 plate, 19.2ms
13: 640x640 1 plate, 19.2ms
14: 640x640 1 plate, 19.2ms
15: 640x640 1 plate, 19.2ms
Speed: 17.1ms preprocess, 19.2ms inference, 6.7ms postprocess per image at shape (1, 3, 640, 640)
Yolo results are generated for batch 0 

Found Bounding Boxes
Images were cropped 

conf=0.5, show=False, save=False
Running inference on Image

0: 640x640 1 plate, 17.3ms
1: 640x640 1 plate, 17.3ms
2: 640x640 1 plate, 17.3ms
3: 640x640 1 plate, 17.3ms
4: 640x640 1 plate, 17.3ms
5: 640x640 2 plates, 17.3ms
6: 640x640 1 plate, 17.3ms
7: 640x640 1 plate, 17.3ms
8: 640x640

In [6]:
output_dir = "C:/Users/Alireza/Desktop/plates/"
IMUtils.save_cropped_images(cropped_out, output_dir)

Saving images: 100%|██████████| 190/190 [00:00<00:00, 689.37file/s]

Files were saved successfully in C:/Users/Alireza/Desktop/plates/ 



# test

(array([0.02825701, 0.09577547]),
 array([0.03425093, 0.86719203]),
 array([0.97101385, 0.8219808 ]),
 array([0.96844503, 0.07316985]))

In [ ]:
dst_points = np.array([[0, 0], [image_width, 0], [image_width, image_height], [0, image_height]], dtype=np.float32)
src_points  = np.array(list(shape_corners.values()), dtype=np.float32)

M = cv2.getPerspectiveTransform(src_points, dst_points)
rectified_image = cv2.warpPerspective(gray_image_1, M, (image_width, image_height))

plt.imshow(rectified_image, cmap='gray')
plt.axis('off')
plt.show()

# test 2

In [1]:
from services.ai_services.my_models import Plate_ResNet
import services.ai_services.utils as AIUtils
from torch.utils.data import DataLoader

In [2]:
device = AIUtils.select_device()
res = Plate_ResNet(device=device)

NVIDIA GeForce GTX 1660 Ti have been located and selected
Selected model is ResNetType.Base
ResNet18
Layer (type:depth-idx)                        Param #
CustomResNet                                  --
├─ResNet: 1-1                                 --
│    └─Conv2d: 2-1                            9,408
│    └─BatchNorm2d: 2-2                       128
│    └─ReLU: 2-3                              --
│    └─MaxPool2d: 2-4                         --
│    └─Sequential: 2-5                        --
│    │    └─BasicBlock: 3-1                   73,984
│    │    └─BasicBlock: 3-2                   73,984
│    └─Sequential: 2-6                        --
│    │    └─BasicBlock: 3-3                   230,144
│    │    └─BasicBlock: 3-4                   295,424
│    └─Sequential: 2-7                        --
│    │    └─BasicBlock: 3-5                   919,040
│    │    └─BasicBlock: 3-6                   1,180,672
│    └─Sequential: 2-8                        --
│    │    └─BasicBlock: 3-7

In [3]:
res.freeze_unfreeze(True)
res.write_summary()

Start freezing feature extractor ...
Feature extractor freezed
Model is in cuda
Layer (type:depth-idx)                        Param #
CustomResNet                                  --
├─ResNet: 1-1                                 --
│    └─Conv2d: 2-1                            (9,408)
│    └─BatchNorm2d: 2-2                       (128)
│    └─ReLU: 2-3                              --
│    └─MaxPool2d: 2-4                         --
│    └─Sequential: 2-5                        --
│    │    └─BasicBlock: 3-1                   (73,984)
│    │    └─BasicBlock: 3-2                   (73,984)
│    └─Sequential: 2-6                        --
│    │    └─BasicBlock: 3-3                   (230,144)
│    │    └─BasicBlock: 3-4                   (295,424)
│    └─Sequential: 2-7                        --
│    │    └─BasicBlock: 3-5                   (919,040)
│    │    └─BasicBlock: 3-6                   (1,180,672)
│    └─Sequential: 2-8                        --
│    │    └─BasicBlock: 3-7     

In [4]:
dataset = AIUtils.CustomResNetDataset("C:/Users/Alireza/Desktop/plate_data/train/images",
                                      "C:/Users/Alireza/Desktop/plate_data/train/labels")
loader = DataLoader(dataset, 16, shuffle=True)

In [8]:
a = loader.__iter__().__next__()

In [9]:
a[1].shape

torch.Size([16, 4, 2])

In [11]:
import torch

In [12]:
a = res.infer(torch.rand([16,3,224,224]))

Running inference on Image Batch
Outputs Generated


In [15]:
a.shape

torch.Size([16, 4, 2])